In [59]:
import os
import torch
import numpy as np
import csv
import math 
import random
import gzip
from os import path

import torch.nn as nn
import torch.nn.functional as F
from scipy.stats import bernoulli
from sklearn import metrics
from torch.utils.data import Dataset, DataLoader

from utils import seq2pad, dinuc_shuffle, complement, reverse_complement, datasets, logsampler, sqrtsampler

print(torch.__version__)
print('using gpu : ', torch.cuda.is_available())

1.10.2
using gpu :  True


In [60]:
# Settings

codetest = True
testingChip = False
num_motif = 16
bases = 'ACGT' # DNA bases
# basesRNA = 'ACGU' # RNA bases
dictReverse = {'A':'T','C':'G','G':'C','T':'A','N':'N'} #dictionary to implement reverse-complement mode
reverse_mode=False
# Device Configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Hyperparameters
epochs = 5
num_classes = 10
batch_size = 100
lr = 0.001
# dataset
path = './data/encode/'
dataset_names = datasets(path)

In [61]:
# testing seq2pad
if(codetest):
    print("\n"+"###"*20)
    print("testing seq2pad")
    seq_test = 'ATGGATGG'
    motiflen_test = 3
    s = seq2pad(seq_test, motiflen_test)
    print(s)
    print("###"*20)

# testing dinuc_shuffle
if(codetest):
    print("\n"+"###"*20)
    print("testing dinuc_shuffle")
    print(dinuc_shuffle(seq_test))
    print("###"*20)

# testing complement
if(codetest):
    print("\n"+"###"*20)
    print("testing complement")
    print(complement(seq_test))
    print(list(seq_test))
    print("###"*20)

# testing reverse_complement
if(codetest):
    print("\n"+"###"*20)
    print("testing reverse_complement")
    print(reverse_complement(seq_test))
    print("###"*20)

# testing datasets
if(codetest):
    print("\n"+"###"*20)
    print("testing datasets")
    for i in range(len(dataset_names[0])):
        print(dataset_names[0][i])
        print(dataset_names[1][i])
        print("___"*20)
    print("###"*20)


############################################################
testing seq2pad
[[0.25 0.25 1.   0.   0.   0.   1.   0.   0.   0.   0.25 0.25]
 [0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.25]
 [0.25 0.25 0.   0.   1.   1.   0.   0.   1.   1.   0.25 0.25]
 [0.25 0.25 0.   1.   0.   0.   0.   1.   0.   0.   0.25 0.25]]
############################################################

############################################################
testing dinuc_shuffle
GGGGATAT
############################################################

############################################################
testing complement
['T', 'A', 'C', 'C', 'T', 'A', 'C', 'C']
['A', 'T', 'G', 'G', 'A', 'T', 'G', 'G']
############################################################

############################################################
testing reverse_complement
CCATCCAT
############################################################

############################################################
testing d

In [62]:
class Chip():
    def __init__(self, filename, motiflen=24, reverse_complement_mode = reverse_mode):
        self.file = filename
        self.motiflen = motiflen
        self.reverse_complement_mode = reverse_complement_mode
    
    def openFile(self):
        train_dataset = []
        with gzip.open(self.file, 'rt') as data:
            next(data)
            reader = csv.reader(data, delimiter = '\t')
            if not self.reverse_complement_mode:
                for row in reader:
                    train_dataset.append([seq2pad(row[2], self.motiflen), [1]]) # target = 1
                    train_dataset.append([seq2pad(dinuc_shuffle(row[2]), self.motiflen), [0]]) # target = 0
            else:
                for row in reader:
                      train_dataset.append([seq2pad(row[2],self.motiflen),[1]])
                      train_dataset.append([seq2pad(reverse_complement(row[2]),self.motiflen),[1]])
                      train_dataset.append([seq2pad(dinuc_shuffle(row[2]),self.motiflen),[0]])
                      train_dataset.append([seq2pad(dinuc_shuffle(reverse_complement(row[2])),self.motiflen),[0]])
        
        train_dataset_pad = train_dataset

        size = int(len(train_dataset_pad)/3)
        firstvalid = train_dataset_pad[:size]
        secondvalid = train_dataset_pad[size:2*size]
        thirdvalid = train_dataset_pad[2*size:]
        firsttrain = secondvalid+thirdvalid
        secondtrain = firstvalid+thirdvalid
        thirdtrain = firstvalid+secondvalid

        return firsttrain, firstvalid, secondtrain, secondvalid, thirdtrain, thirdvalid, train_dataset_pad

In [63]:
chipseq = Chip(dataset_names[0][2]) #'./data/encode/ELK1_GM12878_ELK1_(1277-1)_Stanford_AC.seq.gz'

In [64]:
train1, valid1, train2, valid2, train3, valid3, all_data = chipseq.openFile()

In [65]:
# testing Chip
if(codetest):
    if(testingChip):
        dataset = []
        with gzip.open(dataset_names[0][2], 'rt') as data:
            next(data)
            reader = csv.reader(data, delimiter='\t')
            for row in reader:
                print(row[2], " : ", len(row[2]))

In [18]:
class chipseq_dataset(Dataset):
    def __init__(self, xy = None):
        self.x_data = np.asarray([element[0] for element in xy], dtype=np.float32)
        self.y_data = np.asarray([element[1] for element in xy], dtype=np.float32)
        self.x_data = torch.from_numpy(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.len = len(self.x_data)
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [19]:
train1_dataset=chipseq_dataset(train1)
train2_dataset=chipseq_dataset(train2)
train3_dataset=chipseq_dataset(train3)
valid1_dataset=chipseq_dataset(valid1)
valid2_dataset=chipseq_dataset(valid2)
valid3_dataset=chipseq_dataset(valid3)
all_data_dataset=chipseq_dataset(all_data)

In [20]:
if(codetest):
    train2_test = []
    with gzip.open(dataset_names[0][2], 'rt') as data:
        next(data)
        reader = csv.reader(data, delimiter = '\t')
        for row in reader:
            train2_test.append([row[2], [1]]) # target = 1
            train2_test.append([dinuc_shuffle(row[2]), [0]]) # target = 0
    
    print(train2_test[0]) # -> the first data on 'data/encode/ELK1_GM12878_ELK1_(1277-1)_Stanford_AC.seq.gz'
    # to see the data, please refer to the copy of it, 'data/encode/ELK1_GM12878_ELK1_(1277-1)_Stanford_AC.seq copy'

# portion of the output of train2_dataset.__gettiem__(0) -> 'CCCG'...
    # 0.0000, 0.0000, 0.0000, 0.0000
    # 1.0000, 1.0000, 1.0000, 0.0000
    # 0.0000, 0.0000, 0.0000, 1.0000
    # 0.0000, 0.0000, 0.0000, 0.0000

# please note that the output tensor is transposed

['CCCGCTCCCTAGCAACGATTGGTTAACGAGGCCTGGTTTCCAGAAATGGGCACTATTTCCGGACGATGTTTGTAGAAGGGAGATCGCTGGGCGGGCGGACT', [1]]


In [21]:
if reverse_mode:
  train_loader1 = DataLoader(dataset=train1_dataset,batch_size=batch_size,shuffle=False)
  train_loader2 = DataLoader(dataset=train2_dataset,batch_size=batch_size,shuffle=False)
  train_loader3 = DataLoader(dataset=train3_dataset,batch_size=batch_size,shuffle=False)
  valid1_loader = DataLoader(dataset=valid1_dataset,batch_size=batch_size,shuffle=False)
  valid2_loader = DataLoader(dataset=valid2_dataset,batch_size=batch_size,shuffle=False)
  valid3_loader = DataLoader(dataset=valid3_dataset,batch_size=batch_size,shuffle=False)
  alldataset_loader=DataLoader(dataset=all_data_dataset,batch_size=batch_size,shuffle=False)
else:
  train_loader1 = DataLoader(dataset=train1_dataset,batch_size=batch_size,shuffle=True)
  train_loader2 = DataLoader(dataset=train2_dataset,batch_size=batch_size,shuffle=True)
  train_loader3 = DataLoader(dataset=train3_dataset,batch_size=batch_size,shuffle=True)
  valid1_loader = DataLoader(dataset=valid1_dataset,batch_size=batch_size,shuffle=False)
  valid2_loader = DataLoader(dataset=valid2_dataset,batch_size=batch_size,shuffle=False)
  valid3_loader = DataLoader(dataset=valid3_dataset,batch_size=batch_size,shuffle=False)
  alldataset_loader=DataLoader(dataset=all_data_dataset,batch_size=batch_size,shuffle=False)

train_dataloader = [train_loader1, train_loader2, train_loader3]
valid_dataloader = [valid1_loader, valid2_loader, valid3_loader]

In [22]:
class ConvNet(nn.Module):
    def __init__(self, num_motif, motif_len, poolType, neuType, mode, droprate, lr, momentum_rate, 
                sigmaConv, sigmaNeu, beta1, beta2, beta3, reverse_complement_mode = reverse_mode):
        super(ConvNet, self).__init__()
        self.poolType = poolType
        self.neuType = neuType
        self.mode = mode
        self.reverse_complement_mode = reverse_complement_mode
        self.droprate = droprate
        self.lr = lr
        self.momentum_rate = momentum_rate
        self.sigmaConv = sigmaConv
        self.sigmaNeu = sigmaNeu
        self.beta1 = beta1
        self.beta2 = beta2
        self.beta3 = beta3

        self.wConv = torch.randn(num_motif, 4, motif_len).to(device)
        torch.nn.init.normal_(self.wConv, mean = 0, std = self.sigmaConv)
        self.wConv.requires_grad = True

        self.wRect = torch.randn(num_motif).to(device)
        torch.nn.init.normal_(self.wRect)
        self.wRect = -self.wRect
        self.wRect.requires_grad = True

        if neuType == 'nohidden':
            if poolType == 'maxavg':
                self.wNeu = torch.randn(2*num_motif, 1).to(device)
            else:
                self.wNeu = torch.randn(num_motif, 1).to(device)
            self.wNeuBias = torch.randn(1).to(device)
            torch.nn.init.normal_(self.wNeu, mean=0, std = self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias, mean=0, std = self.sigmaNeu)
        
        else:
            if poolType == 'maxavg':
                self.wHidden = torch.randn(2*num_motif, 32).to(device)
            else:
                self.wHidden = torch.randn(num_motif, 32).to(device)
            self.wHiddenBias = torch.randn(32).to(device)
            self.wNeu = torch.randn(32, 1).to(device)
            self.wNeuBias = torch.randn(1).to(device)
            torch.nn.init.normal_(self.wNeu, mean=0, std = self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias, mean=0, std = self.sigmaNeu)
            torch.nn.init.normal_(self.wHidden, mean=0, std = 0.3)
            torch.nn.init.normal_(self.wHiddenBias, mean=0, std = 0.3)

            self.wHidden.requires_grad = True
            self.wHiddenBias.requires_grad = True
        
        self.wNeu.requires_grad = True
        self.wNeuBias.requires_grad = True
    
    def divide_two_tensors(self, x):
        l = torch.unbind(x)

        list1 = [l[2*i] for i in range(int(x.shape[0]/2))]
        list2 = [l[2*i + 1] for i in range(int(x.shape[0]/2))]
        x1 = torch.stack(list1, 0)
        x2 = torch.stack(list2, 0)
        return x1, x2
    
    def forward_pass(self, x, mask=None, use_mask=False):
        conv = F.conv1d(x, self.wConv, bias = self.wRect, stride = 1, padding=0)
        rect = conv.clamp(min=0) # -> rectification
        # global out # ???? 
        maxPool, _ = torch.max(rect, dim=2)
        if self.poolType == 'maxavg':
            avgPool = torch.mean(rect, dim=2)
            pool = torch.cat((maxPool, avgPool), 1) # -> pool 순서가 논문과 다르지만 상관 없을것 같다!
        else:
            pool = maxPool
        if(self.neuType == 'nohidden'):
            if self.mode == 'training':
                if not use_mask:
                    mask = bernoulli.rvs(self.droprate, size=len(pool[0]))
                    mask = torch.from_numpy(mask).float().to(device)
                pooldrop = pool*mask
                out = pooldrop @ self.wNeu
                out.add_(self.wNeuBias)
            else:
                out = self.droprate*(pool @ self.wNeu)
                out.add_(self.wNeuBias)
        else:
            hid = pool @ self.wHidden
            hid.add_(self.wHiddenBias)
            hid = hid.clamp(min = 0) # rectification
            if self.mode == 'training':
                if not use_mask:
                    mask = bernoulli.rvs(self.droprate, size = len(hid[0]))
                    mask = torch.from_numpy(mask).float().to(device)
                hid_drop = hid*mask
                out = self.droprate*(hid@self.wNeu)
                out.add_(self.wNeuBias)
            else:
                out = self.droprate*(hid@self.wNeu)
                out.add_(self.wNeuBias)
        
        return out, mask
    
    def forward(self, x):
        if not self.reverse_complement_mode:
            out, _ = self.forward_pass(x)
        else:
            x1, x2 = self.divide_two_tensors(x)
            out1, mask = self.forward_pass(x1)
            out2, _ = self.forward_pass(x2, mask, True)
            out = torch.max(out1, out2)
        
        return out


In [23]:
AUC_best = 0
learning_steps_list = [4000, 8000, 12000, 16000, 20000]

for number in range(5):
    pool_list = ['max', 'maxavg']
    random_pool = random.choice(pool_list)

    neuType_list = ['hidden', 'nohidden']
    random_neuType = random.choice(neuType_list)

    dropout_list = [0.5, 0.75, 1.0]
    drop_rate = random.choice(dropout_list)

    lr = logsampler(0.0005, 0.05)
    momentum_rate = sqrtsampler(0.95, 0.99)
    sigmaConv = logsampler(10**-7, 10**-3)
    sigmaNeu=logsampler(10**-5,10**-2) 
    beta1=logsampler(10**-15,10**-3)
    beta2=logsampler(10**-10,10**-3)
    beta3=logsampler(10**-10,10**-3)

    model_AUC = [[], [], []]

    for kk in range(3):
        model  = ConvNet(16, 24, random_pool, random_neuType, 'training', drop_rate, lr, momentum_rate, sigmaConv, sigmaNeu, beta1, beta2, beta3, reverse_complement_mode=reverse_mode).to(device)
        if random_neuType == 'nohidden':
            optimizer = torch.optim.SGD([model.wConv, model.wRect, model.wNeu, model.wNeuBias], lr = model.lr, momentum=model.momentum_rate, nesterov=True)
        else:
            optimizer = torch.optim.SGD([model.wConv, model.wRect, model.wNeu, model.wNeuBias, model.wHidden, model.wHiddenBias], lr = model.lr, momentum=model.momentum_rate, nesterov=True)
        
        train_loader = train_dataloader[kk]
        valid_loader = valid_dataloader[kk]

        learning_steps = 0
        while learning_steps <= 20000:
            model.mode = 'training'
            auc = []
            for i, (data, target) in enumerate(train_loader):
                data = data.to(device)
                target = target.to(device)
                if model.reverse_complement_mode:
                    target_2 = torch.randn(int(target.shape[0]/2), 1) # 뭐하는 부분인지 이해 안됨!! -> target.shape[0] = 64
                    for i in range(target_2.shape[0]):
                        target_2[i] = target[2*i]
                    target = target_2.to(device)
                
                # Forward Pass
                output = model(data)
                if model.neuType == 'nohidden':
                    loss = F.binary_cross_entropy(torch.sigmoid(output), target) + model.beta1*model.wConv.norm() + model.beta3*model.wNeu.norm()
                else: 
                    loss = F.binary_cross_entropy(torch.sigmoid(output), target) + model.beta1*model.wConv.norm() + model.beta2*model.wHidden.norm() + model.beta3*model.wNeu.norm()
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                learning_steps+=1

                if learning_steps%4000 == 0:
                    
                    with torch.no_grad():
                        model.mode = 'test'
                        auc = []
                        for i, (data, target) in enumerate(valid_loader):
                            data = data.to(device)
                            target = target.to(device)
                            if model.reverse_complement_mode:
                                target_2 = torch.randn(int(target.shape[0]/2), 1)
                                for i in range(target_2.shape[0]):
                                    target_2[i] = target[2*i]
                                target = target_2.to(device)
                            # Forward Pass
                            output = model(data)
                            pred_sig = torch.sigmoid(output)
                            pred = pred_sig.cpu().detach().numpy().reshape(output.shape[0])
                            labels = target.cpu().numpy().reshape(output.shape[0])

                            auc.append(metrics.roc_auc_score(labels, pred))

                        model_AUC[kk].append(np.mean(auc))
                        print('AUC performance of traing fold-', kk+1, ' with learning steps -', learning_steps_list[len(model_AUC[kk])-1], " : ", np.mean(auc))
    
    print('---'*5)
    for n in range(5):
        AUC = (model_AUC[0][n] + model_AUC[1][n] + model_AUC[2][n])/3
        if (AUC > AUC_best):
            AUC_best = AUC
            best_learning_steps = learning_steps_list[n]
            best_lr = model.lr
            best_momentum = model.momentum_rate
            best_neuType = model.neuType
            best_poolType = model.poolType
            best_sigmaConv = model.sigmaConv
            best_droprate = model.droprate
            best_sigmaNeu = model.sigmaNeu
            best_beta1 = model.beta1
            best_beta2 = model.beta2
            best_beta3 = model.beta3

print('best_poolType=',best_poolType)
print('best_neuType=',best_neuType)
print('best_AUC=',AUC_best)
print('best_learning_steps=',best_learning_steps)      
print('best_LearningRate=',best_lr)
print('best_LearningMomentum=',best_momentum)
print('best_sigmaConv=',best_sigmaConv)
print('best_dropprob=',best_droprate)
print('best_sigmaNeu=',best_sigmaNeu)
print('best_beta1=',best_beta1)
print('best_beta2=',best_beta2)
print('best_beta3=',best_beta3)

best_hyperparameters = {'best_poolType': best_poolType,'best_neuType':best_neuType,'best_learning_steps':best_learning_steps,'best_LearningRate':best_lr,
                        'best_LearningMomentum':best_momentum,'best_sigmaConv':best_sigmaConv,'best_dropprob':best_droprate,
                        'best_sigmaNeu':best_sigmaNeu,'best_beta1':best_beta1, 'best_beta2':best_beta2,'best_beta3':best_beta3}

AUC performance of traing fold- 1  with learning steps - 4000  :  0.8977259655377301
AUC performance of traing fold- 1  with learning steps - 8000  :  0.891599881164587
AUC performance of traing fold- 1  with learning steps - 12000  :  0.8900002376708258
AUC performance of traing fold- 1  with learning steps - 16000  :  0.8899142008318479
AUC performance of traing fold- 1  with learning steps - 20000  :  0.8855037433155081
AUC performance of traing fold- 2  with learning steps - 4000  :  0.8104884135472372
AUC performance of traing fold- 2  with learning steps - 8000  :  0.7990326797385621
AUC performance of traing fold- 2  with learning steps - 12000  :  0.8038925133689839
AUC performance of traing fold- 2  with learning steps - 16000  :  0.8017612596553773
AUC performance of traing fold- 2  with learning steps - 20000  :  0.8010915626856803
AUC performance of traing fold- 3  with learning steps - 4000  :  0.7733527959331881
AUC performance of traing fold- 3  with learning steps - 800

In [46]:
# Save Hyperparameters

name = dataset_names[0][2]
name = name.split(path)[1].split("_AC")[0]
torch.save(best_hyperparameters, './Hyperparameters/'+name+'.pth')

In [47]:
class ConvNet_test(nn.Module):
    def __init__(self,nummotif,motiflen,poolType,neuType,mode,dropprob, learning_rate,learning_Momentum,sigmaConv,sigmaNeu,beta1,beta2,beta3,learning_steps, reverse_complemet_mode):
        super(ConvNet_test, self).__init__()
        self.poolType=poolType
        self.neuType=neuType
        self.mode=mode
        self.learning_rate=learning_rate
        self.reverse_complemet_mode=reverse_complemet_mode
        self.momentum_rate=learning_Momentum
        self.sigmaConv=sigmaConv
        self.wConv=torch.randn(nummotif,4,motiflen).to(device)
        torch.nn.init.normal_(self.wConv,mean=0,std=self.sigmaConv)
        self.wConv.requires_grad=True
        self.wRect=torch.randn(nummotif).to(device)
        torch.nn.init.normal_(self.wRect)
        self.wRect=-self.wRect
        self.wRect.requires_grad=True
        self.dropprob=dropprob
        self.sigmaNeu=sigmaNeu
        self.wHidden=torch.randn(2*nummotif,32).to(device)
        self.wHiddenBias=torch.randn(32).to(device)
        if neuType=='nohidden':
            
            if poolType=='maxavg':
                self.wNeu=torch.randn(2*nummotif,1).to(device)
            else:
                self.wNeu=torch.randn(nummotif,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)

        else:
            if poolType=='maxavg':
                self.wHidden=torch.randn(2*nummotif,32).to(device)
            else:
                
                self.wHidden=torch.randn(nummotif,32).to(device)
            self.wNeu=torch.randn(32,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            self.wHiddenBias=torch.randn(32).to(device)
            torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wHidden,mean=0,std=0.3)
            torch.nn.init.normal_(self.wHiddenBias,mean=0,std=0.3)
            
  
            self.wHidden.requires_grad=True
            self.wHiddenBias.requires_grad=True
            #wHiddenBias=tf.Variable(tf.truncated_normal([32,1],mean=0,stddev=sigmaNeu)) #hidden bias for everything

        self.wNeu.requires_grad=True
        self.wNeuBias.requires_grad=True
        

        self.beta1=beta1
        self.beta2=beta2
        self.beta3=beta3
        

    
    def divide_two_tensors(self,x):
        l=torch.unbind(x)

        list1=[l[2*i] for i in range(int(x.shape[0]/2))]
        list2=[l[2*i+1] for i in range(int(x.shape[0]/2))]
        x1=torch.stack(list1,0)
        x2=torch.stack(list2,0)
        return x1,x2
    def forward_pass(self,x,mask=None,use_mask=False):
        
        conv=F.conv1d(x, self.wConv, bias=self.wRect, stride=1, padding=0)
        rect=conv.clamp(min=0)
        maxPool, _ = torch.max(rect, dim=2)
        if self.poolType=='maxavg':
            avgPool= torch.mean(rect, dim=2)                          
            pool=torch.cat((maxPool, avgPool), 1)
        else:
            pool=maxPool
        if(self.neuType=='nohidden'):
            if self.mode=='training': 
                if  not use_mask:
                    mask=bernoulli.rvs(self.dropprob, size=len(pool[0]))
                    mask=torch.from_numpy(mask).float().to(device)
                pooldrop=pool*mask
                out=pooldrop @ self.wNeu
                out.add_(self.wNeuBias)
            else:
                out=self.dropprob*(pool @ self.wNeu)
                out.add_(self.wNeuBias)       
        else:
            hid=pool @ self.wHidden
            hid.add_(self.wHiddenBias)
            hid=hid.clamp(min=0)
            if self.mode=='training': 
                if  not use_mask:
                    mask=bernoulli.rvs(self.dropprob, size=len(hid[0]))
                    mask=torch.from_numpy(mask).float().to(device)
                hiddrop=hid*mask
                out=self.dropprob*(hid @ self.wNeu)
                out.add_(self.wNeuBias)
            else:
                out=self.dropprob*(hid @ self.wNeu)
                out.add_(self.wNeuBias) 
        return out,mask
       
    def forward(self, x):
        
        if not  self.reverse_complemet_mode:
            out,_=self.forward_pass(x)
        else:
            
            x1,x2=self.divide_two_tensors(x)
            out1,mask=self.forward_pass(x1)
            out2,_=self.forward_pass(x2,mask,True)
            out=torch.max(out1, out2)
        
        return out

In [53]:
AUC_best = 0
learning_steps_list=[4000,8000,12000,16000,20000]

best_hyperparameters = torch.load('./Hyperparameters/'+name+'.pth')

best_poolType=best_hyperparameters['best_poolType']
best_neuType=best_hyperparameters['best_neuType']
best_learning_steps=best_hyperparameters['best_learning_steps']
best_lr=best_hyperparameters['best_LearningRate']
best_droprate=best_hyperparameters['best_dropprob']
best_momentum=best_hyperparameters['best_LearningMomentum']
best_sigmaConv=best_hyperparameters['best_sigmaConv']
best_sigmaNeu=best_hyperparameters['best_sigmaNeu']
best_beta1=best_hyperparameters['best_beta1']
best_beta2=best_hyperparameters['best_beta2']
best_beta3=best_hyperparameters['best_beta3']

for number_models in range(6):
    model = ConvNet_test(16, 24, best_poolType, best_neuType, 'training', best_lr, best_momentum, best_sigmaConv, best_droprate, best_sigmaNeu, best_beta1, best_beta2, best_beta3, best_learning_steps, False).to(device)

    if model.neuType == 'nohidden':
        optimizer = torch.optim.SGD([model.wConv, model.wRect, model.wNeu, model.wNeuBias], lr = model.learning_rate, momentum= model.momentum_rate, nesterov=True)
    else:
        optimizer = torch.optim.SGD([model.wConv, model.wRect, model.wNeu, model.wNeuBias, model.wHidden, model.wHiddenBias], lr = model.learning_rate, momentum=model.momentum_rate, nesterov=True)
    
    train_loader = alldataset_loader
    valid_loader = alldataset_loader
    learning_steps = 0

    while learning_steps <= best_learning_steps:
        for i, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)
            if reverse_mode:
                target_2 = torch.randn(int(target.shape[0]/2), 1)
                for i in range(target_2.shape[0]):
                    target_2[i] = target[2*i]
                target = target_2.to(device)
            
            # Forward Pass
            output = model(data)
            if model.neuType == 'nohidden':
                loss = F.binary_cross_entropy(torch.sigmoid(output), target) + model.beta1*model.wConv.norm() + model.beta3*model.wNeu.norm()
            else: 
                loss = F.binary_cross_entropy(torch.sigmoid(output), target) + model.beta1*model.wConv.norm() + model.beta2*model.wHidden.norm() + model.beta3*model.wNeu.norm()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            learning_steps += 1
    
    with torch.no_grad():
        model.mode = 'test'
        auc = []
        for i, (data, target) in enumerate(valid_loader):
            data = data.to(device)
            target = target.to(device)
            if reverse_mode:
                target_2 = torch.randn(int(target.shape[0]/2), 1)
                for i in range(target_2.shape[0]):
                    target_2[i] = target[2*i]
                target = target_2.to(device)
            
            # Forward Pass
            output = model(data)
            pred_sig = torch.sigmoid(output)
            pred = pred_sig.cpu().detach().numpy().reshape(output.shape[0])
            labels = target.cpu().numpy().reshape(output.shape[0])

            auc.append(metrics.roc_auc_score(labels, pred))
        
        AUC_training = np.mean(auc)
        print('AUC for model ', number_models, ' = ', AUC_training)
        if AUC_training > AUC_best:
            state = {'conv': model.wConv,'rect':model.wRect,'wHidden':model.wHidden,'wHiddenBias':model.wHiddenBias,'wNeu':model.wNeu,'wNeuBias':model.wNeuBias}
            torch.save(state, './Models/'+name+'.pth')


AUC for model  0  =  0.9190644548476828
AUC for model  1  =  0.9139054888391344
AUC for model  2  =  0.9134848632878756
AUC for model  3  =  0.9212003392360404
AUC for model  4  =  0.9223371192075447
AUC for model  5  =  0.9161103738381164


In [56]:
checkpoint = torch.load('./Models'+name+'.pth')
model = ConvNet_test(16, 24, best_poolType, best_neuType, 'test', best_lr, best_momentum, best_sigmaConv, best_droprate, best_sigmaNeu, best_beta1, best_beta2, best_beta3, best_learning_steps, reverse_mode).to(device)
model.wConv=checkpoint['conv']
model.wRect=checkpoint['rect']
model.wHidden=checkpoint['wHidden']
model.wHiddenBias=checkpoint['wHiddenBias']
model.wNeu=checkpoint['wNeu']
model.wNeuBias=checkpoint['wNeuBias']

with torch.no_grad():
    model.mode = 'test'
    auc = []

    for i, (data, target) in enumerate(valid_loader):
        data = data.to(device)
        target = target.to(device)
        if reverse_mode:
            target_2 = torch.randn(int(target.shape[0]/2), 1)
            for i in range(target_2.shape[0]):
                target_2[i] = target[2*i]
            target = target_2.to(device)
        
        # Forward Pass
        output = model(data)
        pred_sig = torch.sigmoid(output)
        pred = pred_sig.cpu().detach().numpy().reshape(output.shape[0])
        labels = target.cpu().numpy().reshape(output.shape[0])

        auc.append(metrics.roc_auc_score(labels, pred))

    AUC_training = np.mean(auc)
    print(AUC_training)

0.9161103738381164


In [57]:
class Chip_test():
    def __init__(self,filename,motiflen=24,reverse_complemet_mode=reverse_mode):
        self.file = filename
        self.motiflen = motiflen
        self.reverse_complemet_mode=reverse_complemet_mode
            
    def openFile(self):
        test_dataset=[]
        with gzip.open(self.file, 'rt') as data:
            next(data)
            reader = csv.reader(data,delimiter='\t')
            if not self.reverse_complemet_mode:
              for row in reader:
                      test_dataset.append([seq2pad(row[2],self.motiflen),[int(row[3])]])
            else:
              for row in reader:
                      test_dataset.append([seq2pad(row[2],self.motiflen),[int(row[3])]])
                      test_dataset.append([seq2pad(reverse_complement(row[2]),self.motiflen),[int(row[3])]])
            
        return test_dataset

In [58]:
chipseq_test=Chip_test(dataset_names[1][2])
test_data=chipseq_test.openFile()
test_dataset=chipseq_dataset(test_data)
batchSize=test_dataset.__len__()
test_loader = DataLoader(dataset=test_dataset,batch_size=batchSize,shuffle=False)

with torch.no_grad():
      model.mode='test'
      auc=[]
     
      for i, (data, target) in enumerate(test_loader):
          data = data.to(device)
          target = target.to(device)
          if reverse_mode:
              target_2=torch.randn(int(target.shape[0]/2),1)
              for i in range(target_2.shape[0]):
                target_2[i]=target[2*i]
              target=target_2.to(device)
          # Forward pass
          output = model(data)
          pred_sig=torch.sigmoid(output)
          pred=pred_sig.cpu().detach().numpy().reshape(output.shape[0])
          labels=target.cpu().numpy().reshape(output.shape[0])
          
          auc.append(metrics.roc_auc_score(labels, pred))
  #                         
      AUC_training=np.mean(auc)
      print('AUC on test data = ',AUC_training)

AUC on test data =  0.904928
